In [1]:
import scispacy
import spacy
import json
import numpy as np

In [2]:
nlp = spacy.load("en_core_sci_lg") 
# TODO - Load Biobert model for medical dataset

In [32]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [3]:
with open("research_papers_cleaned.json") as file:
    papers  = json.load(file)

In [31]:
len(papers)

19815

In [5]:
summary = [paper[2] for paper in papers]
abstracts = [paper[1] for paper in papers]
headings = [paper[0] for paper in papers]

In [6]:
headings[5]

'Identification and Characterization of a Novel Non-Structural Protein of Bluetongue Virus'

In [7]:
abstracts[5]

'Bluetongue virus (BTV) is the causative agent of a major disease of livestock (bluetongue). For over two decades, it has been widely accepted that the 10 segments of the dsRNA genome of BTV encode for 7 structural and 3 non-structural proteins. The non-structural proteins (NS1, NS2, NS3/NS3a) play different key roles during the viral replication cycle. In this study we show that BTV expresses a fourth non-structural protein (that we designated NS4) encoded by an open reading frame in segment 9 overlapping the open reading frame encoding VP6. NS4 is 77-79 amino acid residues in length and highly conserved among several BTV serotypes/strains. NS4 was expressed early post-infection and localized in the nucleoli of BTV infected cells. By reverse genetics, we showed that NS4 is dispensable for BTV replication in vitro, both in mammalian and insect cells, and does not affect viral virulence in murine models of bluetongue infection. Interestingly, NS4 conferred a replication advantage to BTV

In [120]:
import torch


In [118]:
data = pd.read_csv('df_doc_vectors.csv')

In [12]:
max_scores_index = scores.argsort()[-10:][::-1]

In [13]:
scores[max_scores_index]

array([0.78404272, 0.78086513, 0.77120374, 0.77085118, 0.76827728,
       0.76737892, 0.764756  , 0.76350952, 0.7584256 , 0.75745367])

In [14]:
for index in max_scores_index:
    print(abstracts[index])

The human coronaviruses (CoV) include HCoV-229E, HCoV-OC43, HCoV-NL63, and HCoV-HKU1, some of which have been known for decades. The severe acute respiratory syndrome (SARS) CoV briefly emerged into the human population but was controlled. In 2012, another novel severely human pathogenic CoV-the Middle East Respiratory Syndrome (MERS)-CoV-was identified in the Kingdom of Saudi Arabia; 80% of over 2000 human cases have been recorded over five years. Targeted research remains key to developing control strategies for MERS-CoV, a cause of mild illness in its camel reservoir. A new therapeutic toolbox being developed in response to MERS is also teaching us more about how CoVs cause disease. Travel-related cases continue to challenge the world's surveillance and response capabilities, and more data are needed to understand unexplained primary transmission. Signs of genetic change have been recorded, but it remains unclear whether there is any impact on clinical disease. How camels came to ca

In [15]:
import torch


In [18]:
from transformers import BertForQuestionAnswering

model = BertForQuestionAnswering.from_pretrained('./scibert_scivocab_uncased/')

In [21]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('./scibert_scivocab_uncased/')

In [30]:
ids = tokenizer.encode('Coronavirus cases are increasing drastically in America')
tokens= tokenizer.convert_ids_to_tokens(ids)
tokens 
# so Berttokenizer can take one or two arguments as an input string like in our case it is taking question and answer_text
# as an input and returing tokens of combined input, so to separate both pair of strings we use sep_ids which separate both by
# ['SEP'] token where left one is question and right one is answer
# then we create array of num_of_Seg_a*0 + num_of_Seg_b*1 where num_of_seg_a = seg_index+1 and num_of_Seg_b = len(array)-num_of_seg_a
# then pass two inputs in model first is input_ids and second is seg_ids = num_of_Seg_a*[0]+num_of_seg_b*[1]
# then we will get start_score and end_score where token[start_score] will give first token of answer and remaining tokens of answer
# can be retrieved easily where indexes varies from start_score+1.......end_score, then check whether there is '##' or not, if 
# two hashes occur before any token then combine text after '##' with previous token as '##' indicates combination of two tokens,
# else add tokens direct into answer and print it.

# in function pass question and abstract of max_score(remember max_score doesn't always give right answer may be top 2nd or 3rd
# index may give right answer)
# Thankyou

['[CLS]',
 'corona',
 '##virus',
 'cases',
 'are',
 'increasing',
 'drastically',
 'in',
 'america',
 '[SEP]']

In [69]:
def answer_question(question, answer_text):
    '''
    Takes a `question` string and an `answer_text` string (which contains the
    answer), and identifies the words within the `answer_text` that are the
    answer. Prints them out.
    '''
    
    
    # ======== Tokenize ========
    # Apply the tokenizer to the input text, treating them as a text-pair.
    input_ids = tokenizer.encode(question, answer_text) # encode returns the id's associated with input question while finding
                                                        # answer in document(abstract cleaned summary)

    # Report how long the input sequence is.
    print('Query has {:,} tokens.\n'.format(len(input_ids)))

    # ======== Set Segment IDs ========
    # Search the input_ids for the first instance of the `[SEP]` token.
    sep_index = input_ids.index(tokenizer.sep_token_id) # sep_token_id is used to separate answer and query(question) in document
    print('sep_index',sep_index)
    # The number of segment A tokens includes the [SEP] token istelf.
    num_seg_a = sep_index +1

    # The remainder are segment B.
    num_seg_b = len(input_ids) - num_seg_a

    # Construct the list of 0s and 1s.
    segment_ids = [0]*num_seg_a + [1]*num_seg_b
    print('num_seg_a',num_seg_a,'num_seg_b',num_seg_b,'segment_ids',segment_ids)
    print('length_input_ids',len(input_ids),'length of segment_ids',len(segment_ids))
    # There should be a segment_id for every input token.
    assert len(segment_ids) == len(input_ids)

    # ======== Evaluate ========
    # Run our example question through the model.
    start_scores, end_scores = model(torch.tensor([input_ids]), # The tokens representing our input text.
                                    token_type_ids=torch.tensor([segment_ids])) # The segment IDs to differentiate question from answer_text

    # ======== Reconstruct Answer ========
    # Find the tokens with the highest `start` and `end` scores.
    answer_start = torch.argmax(start_scores)
    answer_end = torch.argmax(end_scores)
    print('answer_start',answer_start,'answer_end',answer_end)
    # Get the string versions of the input tokens.
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    print('tokens',tokens)
    # Start with the first token.
    answer = tokens[answer_start]
    print('first_token_of_answer',answer)
    # Select the remaining answer tokens and join them with whitespace.
    for i in range(answer_start + 1, answer_end + 1):
        
        # If it's a subword token, then recombine it with the previous token.
        if tokens[i][0:2] == '##':
            answer += tokens[i][2:]
        
        # Otherwise, add a space then the token.
        else:
            answer += ' ' + tokens[i]
            print('anwer_tokens',answer)

    print('Answer: "' + answer + '"')

In [99]:
import pandas as pd
# loading top 10 best_docs csv file
top_10_docs = pd.read_csv('best_docs_diff.csv')
top_docs=top_10_docs['best_docs_diff']

In [114]:
question = "what is coronavirus?"
answer_question(question, top_docs[0]) # here top_docs represents answer_text from which we have to find a relevant answer.

Query has 182 tokens.

sep_index 6
num_seg_a 7 num_seg_b 175 segment_ids [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
length_input_ids 182 length of segment_ids 182
answer_start tensor(50) answer_end tensor(135)
tokens ['[CLS]', 'what', 'is', 'corona', '##virus', '?', '[SEP]', 'migration', 'has', 'deep', 'historical', 'roots', 'in', 'south', 'and', 'southern', 'africa', 'and', 'to', 'this', 'day', 'continues', 'to', 'be', 'highly', 'prevalent', 'and', 'a', 'major', 'factor', 'shaping', 'south'